In [1]:
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "1"   #(xxxx is your specific GPU ID)

In [2]:
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.layers import Activation, Dense, Flatten, GlobalAveragePooling2D
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Sequential
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os
from sklearn.preprocessing import MinMaxScaler
import seaborn as sns
from PIL import Image
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from MyEarlyStopping import MyEarlyStopping
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
# from keras.optimizers import adam

import timeit
from sklearn.metrics import confusion_matrix
import sklearn.metrics as metrics
from sklearn.preprocessing import LabelEncoder  

# load dataset 

In [3]:
train_number = 5
train_image = 6 #10:20%, 20: 40%, 30:60%

## Training dataset (40%)

In [4]:
train_data = ImageDataGenerator(preprocessing_function=preprocess_input)
train_generator = train_data.flow_from_directory('/data/home/cou/yuli5/semg/Audio_MEL_Data/train',
                                                batch_size=1,
                                                target_size=(224,224),
                                                class_mode='categorical',
                                               shuffle=False)

Found 1714 images belonging to 10 classes.


In [5]:
image_names = train_generator.filenames
image_no = [i.split("/")[1].split("_")[2].split(".")[0] for i in image_names]
image_no = np.array(list(map(int, image_no)))
ALL_participant_class = [i.split("/")[1].split("_")[1] for i in image_names]
ALL_participant_class = np.array(list(map(int, ALL_participant_class)))
command_class = train_generator.classes
All_participant_class = tf.keras.utils.to_categorical(ALL_participant_class-1, num_classes=train_number)
All_command_class = tf.keras.utils.to_categorical(command_class, num_classes=10)
All_command_uniform = All_command_class*0+1/10

In [6]:
All_Inputs = [next(train_generator)[0][0] for _ in range(len(train_generator))]
All_Inputs = np.array(All_Inputs)

In [7]:
select_indexs_train = image_no<train_image
Train_Inputs = All_Inputs[select_indexs_train]
Train_participant_class = All_participant_class[select_indexs_train]
Train_participant_uniform = Train_participant_class*0+1/train_number
Train_command_class = All_command_class[select_indexs_train]
Train_command_uniform = Train_command_class*0+1/10
#sum(Train_command_class)

## Validation dataset (20%)

In [8]:
val_data = ImageDataGenerator(preprocessing_function=preprocess_input)
val_generator = val_data.flow_from_directory('/data/home/cou/yuli5/semg/Audio_MEL_Data/validation',
                                                batch_size=1,
                                                target_size=(224,224),
                                                class_mode='categorical',
                                               shuffle=False)

Found 543 images belonging to 10 classes.


In [9]:
image_names = val_generator.filenames
participant_class = [i.split("/", 1)[1].split("_")[1] for i in image_names]
participant_class = np.array(list(map(int, participant_class)))
command_class = val_generator.classes
Val_participant_class = tf.keras.utils.to_categorical(participant_class-1, num_classes=train_number)
Val_participant_uniform = Val_participant_class*0+1/train_number
Val_command_class = tf.keras.utils.to_categorical(command_class, num_classes=10)
Val_command_uniform = Val_command_class*0+1/10

In [10]:
Val_Inputs = [next(val_generator)[0][0] for _ in range(len(val_generator))]
Val_Inputs = np.array(Val_Inputs)

## Test dataset (20%)

In [11]:
test_data = ImageDataGenerator(preprocessing_function=preprocess_input)
test_generator = test_data.flow_from_directory('/data/home/cou/yuli5/semg/Audio_MEL_Data/test',
                                                batch_size=1,
                                                target_size=(224,224),
                                                class_mode='categorical',
                                                shuffle=False)

Found 543 images belonging to 10 classes.


In [12]:
image_names = test_generator.filenames
participant_class = [i.split("/", 1)[1].split("_")[1] for i in image_names]
test_unit_participant_class = np.array(list(map(int, participant_class)))
test_unit_command_class = test_generator.classes
Test_participant_class = tf.keras.utils.to_categorical(test_unit_participant_class-1, num_classes=train_number)
Test_participant_uniform = Test_participant_class*0+1/train_number
Test_command_class = tf.keras.utils.to_categorical(test_unit_command_class, num_classes=10)
Test_command_uniform = Test_command_class*0+1/10

In [13]:
Test_Inputs = [next(test_generator)[0][0] for _ in range(len(test_generator))]
Test_Inputs = np.array(Test_Inputs)

In [14]:
Test_Inputs_1 = Test_Inputs[np.where(test_unit_participant_class == 1)]
Test_command_class_1 = Test_command_class[np.where(test_unit_participant_class == 1)]
Test_Inputs_2 = Test_Inputs[np.where(test_unit_participant_class == 2)]
Test_command_class_2 = Test_command_class[np.where(test_unit_participant_class == 2)]
Test_Inputs_3 = Test_Inputs[np.where(test_unit_participant_class == 3)]
Test_command_class_3 = Test_command_class[np.where(test_unit_participant_class == 3)]
Test_Inputs_4 = Test_Inputs[np.where(test_unit_participant_class == 4)]
Test_command_class_4 = Test_command_class[np.where(test_unit_participant_class == 4)]
Test_Inputs_5 = Test_Inputs[np.where(test_unit_participant_class == 5)]
Test_command_class_5 = Test_command_class[np.where(test_unit_participant_class == 5)]

## Speaker 6 Test dataset 

In [15]:
test_data_6 = ImageDataGenerator(preprocessing_function=preprocess_input)
test_generator_6 = test_data_6.flow_from_directory('/data/home/cou/yuli5/semg/Audio_MEL_subject/p_6_split/test',
                                                batch_size=1,
                                                target_size=(224,224),
                                                class_mode='categorical',
                                                shuffle=False)

Found 100 images belonging to 10 classes.


In [16]:
image_names = test_generator_6.filenames
command_class = [i.split("/", 1)[0]for i in image_names]
le = LabelEncoder()
test_unit_command_class_6 = le.fit_transform(command_class)
Test_command_class_6 = tf.keras.utils.to_categorical(test_unit_command_class_6, num_classes=10)

In [17]:
Test_Inputs_6 = [next(test_generator_6)[0][0] for _ in range(len(test_generator_6))]
Test_Inputs_6 = np.array(Test_Inputs_6)

## Speaker 7 Test dataset 

In [18]:
test_data_7 = ImageDataGenerator(preprocessing_function=preprocess_input)
test_generator_7 = test_data_6.flow_from_directory('/data/home/cou/yuli5/semg/Audio_MEL_subject/p_7_split/test',
                                                batch_size=1,
                                                target_size=(224,224),
                                                class_mode='categorical',
                                                shuffle=False)

Found 125 images belonging to 10 classes.


In [19]:
image_names = test_generator_7.filenames
command_class = [i.split("/", 1)[0]for i in image_names]
le = LabelEncoder()
test_unit_command_class_7 = le.fit_transform(command_class)
Test_command_class_7 = tf.keras.utils.to_categorical(test_unit_command_class_7, num_classes=10)

In [20]:
Test_Inputs_7 = [next(test_generator_7)[0][0] for _ in range(len(test_generator_7))]
Test_Inputs_7 = np.array(Test_Inputs_7)

## Speaker 8 Test dataset 

In [21]:
test_data_8 = ImageDataGenerator(preprocessing_function=preprocess_input)
test_generator_8 = test_data_6.flow_from_directory('/data/home/cou/yuli5/semg/Audio_MEL_subject/p_8_split/test',
                                                batch_size=1,
                                                target_size=(224,224),
                                                class_mode='categorical',
                                                shuffle=False)

Found 101 images belonging to 10 classes.


In [22]:
image_names = test_generator_8.filenames
command_class = [i.split("/", 1)[0]for i in image_names]
le = LabelEncoder()
test_unit_command_class_8 = le.fit_transform(command_class)
Test_command_class_8 = tf.keras.utils.to_categorical(test_unit_command_class_8, num_classes=10)

In [23]:
Test_Inputs_8 = [next(test_generator_8)[0][0] for _ in range(len(test_generator_8))]
Test_Inputs_8 = np.array(Test_Inputs_8)

# pd file store performance

In [24]:
# pd file store performance
# Perfomance = pd.DataFrame(columns = ['Model','Time','Partcp_Acc_p15','Command_Acc_p15','Partcp_Acc_p1','Command_Acc_p1',
#                                       'Partcp_Acc_p2','Command_Acc_p2','Partcp_Acc_p3','Command_Acc_p3',
#                                       'Partcp_Acc_p4','Command_Acc_p4','Partcp_Acc_p5','Command_Acc_p5','Acc_p6','Acc_p7','Acc_p8'])

Perfomance = pd.read_csv('Performance_0608_training_Data_Size.csv')
# Perfomance
Perfomance                                                        

,Model,Size,Time,Partcp_Acc_p15,Command_Acc_p15,Partcp_Acc_p1,Command_Acc_p1,Partcp_Acc_p2,Command_Acc_p2,Partcp_Acc_p3,Command_Acc_p3,Partcp_Acc_p4,Command_Acc_p4,Partcp_Acc_p5,Command_Acc_p5,Acc_p6,Acc_p7,Acc_p8
0,Group,10,47.540156,0.9724,0.9484,0.991,0.9550,0.9259,0.8796,0.9652,0.9565,0.9817,0.9633,1.00,0.99,0.67,0.528,0.3366
1,Group,10,48.307249,0.9779,0.9521,1.000,0.9369,0.9352,0.8796,0.9739,0.9652,0.9908,0.9908,0.99,0.99,0.59,0.528,0.2970
2,Group,10,46.201615,0.9613,0.9448,0.991,0.9369,0.9074,0.8611,0.9826,0.9652,0.9358,0.9725,0.99,0.99,0.67,0.432,0.2673
3,Group,10,43.692362,0.9650,0.9521,1.000,0.9459,0.9074,0.8704,0.9826,0.9652,0.9358,0.9908,1.00,0.99,0.62,0.504,0.2673
4,Group,10,44.504580,0.9687,0.9484,0.991,0.9550,0.9259,0.8796,0.9739,0.9652,0.9541,0.9541,1.00,0.99,0.59,0.472,0.3267
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65,Group,10&s1_20&s2_30,53.714297,0.9797,0.9687,0.991,0.9550,1.0000,0.9630,0.9826,0.9565,0.9358,0.9817,0.99,0.99,0.69,0.456,0.3366
66,Group,10&s1_20&s2_30,54.407596,0.9761,0.9742,1.000,0.9730,1.0000,0.9722,0.9739,0.9565,0.9174,0.9817,0.99,0.99,0.68,0.440,0.2673
67,Group,10&s1_20&s2_30,49.085995,0.9779,0.9687,0.991,0.9730,1.0000,0.9537,0.9565,0.9565,0.9541,0.9817,0.99,0.98,0.59,0.440,0.2970
68,Group,10&s1_20&s2_30,51.799617,0.9779,0.9724,0.991,0.9820,1.0000,0.9537,0.9478,0.9565,0.9541,0.9817,1.00,0.99,0.70,0.488,0.2673


# Create model 

In [25]:
actv_fun_1_1 = "relu" 
actv_fun_1_2 = "sigmoid"
shape_1_1 = 128
shape_1_2 = 256
actv_fun_2_1 = "sigmoid" 
actv_fun_2_2 = "sigmoid"
shape_2_1 = 512
shape_2_2 = 512


## Group model 

In [26]:
high_acc = 0
for run in range(0,10):
    # feature extraction layers
    resnet_model = ResNet50(input_shape=(224, 224,3), include_top=False, weights="imagenet")
    for layer in resnet_model.layers:
        layer.trainable = False

    d = resnet_model.output.shape[-1] # dimension of last layer

    ###################### model 1 ###################### 
    layer_1_0 = tf.keras.layers.Dense(d,name="weight_1")(resnet_model.output) #times weight before flatten
    layer_1_1 = tf.keras.layers.Flatten(name='flatten_1')(layer_1_0)

    Dense_1_1 = tf.keras.layers.Dense(shape_1_1, activation=actv_fun_1_1,name='fc1_1')
    layer_1_2 = Dense_1_1(layer_1_1)
    Dense_1_2 = tf.keras.layers.Dense(shape_1_2, activation=actv_fun_1_2,name='fc1_2')
    layer_1_3 = Dense_1_2(layer_1_2)

    Dense_1_3 = tf.keras.layers.Dense(train_number, activation='softmax' ,name='participant_output')
    out_layer_1 = Dense_1_3(layer_1_3)

    ###################### model 2 ###################### 
    layer_2_0 = tf.keras.layers.Dense(d,name="weight_2")(resnet_model.output) #times weight before flatten
    layer_2_1 = tf.keras.layers.Flatten(name='flatten_2')(layer_2_0)

    Dense_2_1 = tf.keras.layers.Dense(shape_2_1, activation=actv_fun_2_1,name='fc2_1')
    layer_2_2  = Dense_2_1(layer_2_1)
    Dense_2_2 = tf.keras.layers.Dense(shape_2_2, activation=actv_fun_2_2,name='fc2_2')
    layer_2_3  = Dense_2_2(layer_2_2)

    Dense_2_3 = tf.keras.layers.Dense(10, activation='softmax',name='command_output')
    out_layer_2 = Dense_2_3(layer_2_3)

    ###################### model 1' ###################### 
    layer_1_2_  = Dense_2_1(layer_1_1)
    layer_1_3_  = Dense_2_2(layer_1_2_)
    out_layer_1_ = Dense_2_3(layer_1_3_)

    ###################### model 1' ###################### 
    layer_2_2_  = Dense_1_1(layer_2_1)
    layer_2_3_  = Dense_1_2(layer_2_2_)
    out_layer_2_ = Dense_1_3(layer_2_3_)

    resnet_model = tf.keras.Model(resnet_model.input, [out_layer_1,out_layer_2,out_layer_1_,out_layer_2_])


    # resnet_model.summary() 

    w_1, w_2, w_1_, w_2_ = 1,1,1,1
    ##################### training ############################
    opt = tf.keras.optimizers.Adam(learning_rate=0.0001)
    callbacks = [MyEarlyStopping(monitor1 = 'val_' + resnet_model.layers[-1].name+'_accuracy',
                                  monitor2 = 'val_' + resnet_model.layers[-2].name+'_accuracy',
                                  patience=5,restore_best_weights=True)]
    resnet_model.compile(optimizer=opt, loss=["categorical_crossentropy","categorical_crossentropy","mse","mse"],
                         loss_weights=[w_1, w_2, w_1_, w_2_], metrics=['accuracy'])

    
    start = timeit.default_timer()
       
    history1 = resnet_model.fit(Train_Inputs, 
                           {resnet_model.layers[-2].name:Train_participant_class, 
                            resnet_model.layers[-1].name:Train_command_class,
                            resnet_model.layers[-1].name+"_1":Train_command_uniform, 
                            resnet_model.layers[-2].name+"_1":Train_participant_uniform}, 
                            validation_data=(Val_Inputs,
                                             {resnet_model.layers[-2].name:Val_participant_class,
                                              resnet_model.layers[-1].name:Val_command_class,
                                              resnet_model.layers[-1].name+"_1":Val_command_uniform,
                                              resnet_model.layers[-2].name+"_1":Val_participant_uniform}), 
                           callbacks=callbacks,
                           batch_size=64,
                           epochs=10)
    
    for layer in resnet_model.layers[0:175]:
        layer.trainable = True
        
    history2 = resnet_model.fit(Train_Inputs, 
                               {resnet_model.layers[-2].name:Train_participant_class, 
                                resnet_model.layers[-1].name:Train_command_class,
                                resnet_model.layers[-1].name+"_1":Train_command_uniform, 
                                resnet_model.layers[-2].name+"_1":Train_participant_uniform}, 
                                validation_data=(Val_Inputs,
                                                 {resnet_model.layers[-2].name:Val_participant_class,
                                                  resnet_model.layers[-1].name:Val_command_class,
                                                  resnet_model.layers[-1].name+"_1":Val_command_uniform,
                                                  resnet_model.layers[-2].name+"_1":Val_participant_uniform}), 
                               callbacks=callbacks,
                               batch_size=64,
                               epochs=10)

    stop = timeit.default_timer()
    run_time = stop - start
    
    ##################### test performance ############################
    predictions = resnet_model.predict(Test_Inputs)[0]
    predicted_classes = np.argmax(predictions, axis=1)+1
    true_classes = test_unit_participant_class
    acc_p15_s = metrics.accuracy_score(true_classes, predicted_classes).round(4)

    predictions = resnet_model.predict(Test_Inputs)[1]
    predicted_classes = np.argmax(predictions, axis=1)
    true_classes = test_unit_command_class
    acc_p15_c = metrics.accuracy_score(true_classes, predicted_classes).round(4)


    # test on p1
    ## speaker classification
    predictions = resnet_model.predict(Test_Inputs_1)[0]
    predicted_classes = np.argmax(predictions, axis=1)+1
    true_classes = np.array([1]*len(predicted_classes))
    acc_p1_s = metrics.accuracy_score(true_classes, predicted_classes).round(4)   
    ## command classification
    predictions = resnet_model.predict(Test_Inputs_1)[1]
    predicted_classes = np.argmax(predictions, axis=1)
    true_classes = np.argmax(Test_command_class_1, axis=-1)
    acc_p1_c = metrics.accuracy_score(true_classes, predicted_classes).round(4)   

    # test on p2
    ## speaker classification
    predictions = resnet_model.predict(Test_Inputs_2)[0]
    predicted_classes = np.argmax(predictions, axis=1)+1
    true_classes = np.array([2]*len(predicted_classes))
    acc_p2_s = metrics.accuracy_score(true_classes, predicted_classes).round(4)   
    ## command classification
    predictions = resnet_model.predict(Test_Inputs_2)[1]
    predicted_classes = np.argmax(predictions, axis=1)
    true_classes = np.argmax(Test_command_class_2, axis=-1)
    acc_p2_c = metrics.accuracy_score(true_classes, predicted_classes).round(4)   

    # test on p3
    ## speaker classification
    predictions = resnet_model.predict(Test_Inputs_3)[0]
    predicted_classes = np.argmax(predictions, axis=1)+1
    true_classes = np.array([3]*len(predicted_classes))
    acc_p3_s = metrics.accuracy_score(true_classes, predicted_classes).round(4)   
    ## command classification
    predictions = resnet_model.predict(Test_Inputs_3)[1]
    predicted_classes = np.argmax(predictions, axis=1)
    true_classes = np.argmax(Test_command_class_3, axis=-1)
    acc_p3_c = metrics.accuracy_score(true_classes, predicted_classes).round(4)   

    # test on p4
    ## speaker classification
    predictions = resnet_model.predict(Test_Inputs_4)[0]
    predicted_classes = np.argmax(predictions, axis=1)+1
    true_classes = np.array([4]*len(predicted_classes))
    acc_p4_s = metrics.accuracy_score(true_classes, predicted_classes).round(4)   
    ## command classification
    predictions = resnet_model.predict(Test_Inputs_4)[1]
    predicted_classes = np.argmax(predictions, axis=1)
    true_classes = np.argmax(Test_command_class_4, axis=-1)
    acc_p4_c = metrics.accuracy_score(true_classes, predicted_classes).round(4)   

    # test on p5
    ## speaker classification
    predictions = resnet_model.predict(Test_Inputs_5)[0]
    predicted_classes = np.argmax(predictions, axis=1)+1
    true_classes = np.array([5]*len(predicted_classes))
    acc_p5_s = metrics.accuracy_score(true_classes, predicted_classes).round(4)   
    ## command classification
    predictions = resnet_model.predict(Test_Inputs_5)[1]
    predicted_classes = np.argmax(predictions, axis=1)
    true_classes = np.argmax(Test_command_class_5, axis=-1)
    acc_p5_c = metrics.accuracy_score(true_classes, predicted_classes).round(4)   

    # test on p6 
    ## command classification
    predictions = resnet_model.predict(Test_Inputs_6)[1]
    predicted_classes = np.argmax(predictions, axis=1)
    true_classes = test_unit_command_class_6
    acc_p6 = metrics.accuracy_score(true_classes, predicted_classes).round(4)                

    # test on p7 
    ## command classification
    predictions = resnet_model.predict(Test_Inputs_7)[1]
    predicted_classes = np.argmax(predictions, axis=1)
    true_classes = test_unit_command_class_7
    acc_p7 = metrics.accuracy_score(true_classes, predicted_classes).round(4)    

    # test on p8 
    ## command classification
    predictions = resnet_model.predict(Test_Inputs_8)[1]
    predicted_classes = np.argmax(predictions, axis=1)
    true_classes = test_unit_command_class_8
    acc_p8 = metrics.accuracy_score(true_classes, predicted_classes).round(4)    

    Perfomance = Perfomance.append({'Model': "Group",'Size':'6','Time':run_time,
                                    'Partcp_Acc_p15':acc_p15_s,'Command_Acc_p15':acc_p15_c,'Partcp_Acc_p1':acc_p1_s,
                                    'Command_Acc_p1':acc_p1_c,'Partcp_Acc_p2':acc_p2_s,'Command_Acc_p2':acc_p2_c,
                                    'Partcp_Acc_p3':acc_p3_s,'Command_Acc_p3':acc_p3_c,'Partcp_Acc_p4':acc_p4_s,
                                    'Command_Acc_p4':acc_p4_c,'Partcp_Acc_p5':acc_p5_s,'Command_Acc_p5':acc_p5_c,
                                    'Acc_p6':acc_p6,'Acc_p7':acc_p7,'Acc_p8':acc_p8}, ignore_index=True)


    if high_acc < acc_p15_c + acc_p15_s:
        resnet_model.save('Initial_group_model_size6_0608.h5')
        high_acc = acc_p15_c + acc_p15_s
        best_index = run
        pd.DataFrame.from_dict(history1.history).to_csv('size6_history1_0608.csv',index=False)
        pd.DataFrame.from_dict(history2.history).to_csv('size6_history2_0608.csv',index=False)
        
    del resnet_model
    run = run + 1

Epoch 1/10
5/5 [==============================] - 4s 752ms/step - loss: 4.0482 - participant_output_loss: 1.6837 - command_output_loss: 2.3531 - command_output_1_loss: 0.0017 - participant_output_1_loss: 0.0098 - participant_output_accuracy: 0.2467 - command_output_accuracy: 0.1567 - command_output_1_accuracy: 0.0333 - participant_output_1_accuracy: 0.3700 - val_loss: 3.5664 - val_participant_output_loss: 1.4340 - val_command_output_loss: 2.1307 - val_command_output_1_loss: 2.8797e-04 - val_participant_output_1_loss: 0.0014 - val_participant_output_accuracy: 0.3886 - val_command_output_accuracy: 0.4254 - val_command_output_1_accuracy: 0.3241 - val_participant_output_1_accuracy: 0.0000e+00
Epoch 2/10
5/5 [==============================] - 1s 277ms/step - loss: 3.3972 - participant_output_loss: 1.3382 - command_output_loss: 2.0578 - command_output_1_loss: 2.2185e-04 - participant_output_1_loss: 0.0010 - participant_output_accuracy: 0.6267 - command_output_accuracy: 0.4333 - command_outpu

Epoch 3/10
5/5 [==============================] - 2s 329ms/step - loss: 1.6819 - participant_output_loss: 0.3926 - command_output_loss: 1.2891 - command_output_1_loss: 4.0301e-05 - participant_output_1_loss: 4.6176e-05 - participant_output_accuracy: 0.9933 - command_output_accuracy: 0.9867 - command_output_1_accuracy: 0.3900 - participant_output_1_accuracy: 0.1067 - val_loss: 1.9114 - val_participant_output_loss: 0.5412 - val_command_output_loss: 1.3702 - val_command_output_1_loss: 3.0801e-05 - val_participant_output_1_loss: 5.4014e-05 - val_participant_output_accuracy: 0.9227 - val_command_output_accuracy: 0.8969 - val_command_output_1_accuracy: 0.2615 - val_participant_output_1_accuracy: 0.1510
Epoch 4/10
5/5 [==============================] - 1s 287ms/step - loss: 1.6003 - participant_output_loss: 0.3639 - command_output_loss: 1.2363 - command_output_1_loss: 3.0542e-05 - participant_output_1_loss: 4.3968e-05 - participant_output_accuracy: 0.9933 - command_output_accuracy: 0.9900 - c

Epoch 5/10
5/5 [==============================] - 2s 303ms/step - loss: 2.7475 - participant_output_loss: 0.9283 - command_output_loss: 1.8189 - command_output_1_loss: 1.4998e-04 - participant_output_1_loss: 1.4018e-04 - participant_output_accuracy: 0.8233 - command_output_accuracy: 0.8633 - command_output_1_accuracy: 0.0000e+00 - participant_output_1_accuracy: 0.1033 - val_loss: 2.7648 - val_participant_output_loss: 0.9424 - val_command_output_loss: 1.8221 - val_command_output_1_loss: 1.1826e-04 - val_participant_output_1_loss: 1.7183e-04 - val_participant_output_accuracy: 0.7827 - val_command_output_accuracy: 0.7624 - val_command_output_1_accuracy: 0.0000e+00 - val_participant_output_1_accuracy: 0.4015
Epoch 6/10
5/5 [==============================] - 2s 346ms/step - loss: 2.5740 - participant_output_loss: 0.8152 - command_output_loss: 1.7585 - command_output_1_loss: 8.3218e-05 - participant_output_1_loss: 1.4145e-04 - participant_output_accuracy: 0.8667 - command_output_accuracy: 0.

Epoch 7/10
5/5 [==============================] - 2s 307ms/step - loss: 1.5022 - participant_output_loss: 0.3246 - command_output_loss: 1.1775 - command_output_1_loss: 2.9347e-05 - participant_output_1_loss: 1.7604e-05 - participant_output_accuracy: 1.0000 - command_output_accuracy: 0.9967 - command_output_1_accuracy: 0.1833 - participant_output_1_accuracy: 0.0733 - val_loss: 1.7694 - val_participant_output_loss: 0.4930 - val_command_output_loss: 1.2764 - val_command_output_1_loss: 3.1645e-05 - val_participant_output_1_loss: 2.6348e-05 - val_participant_output_accuracy: 0.9392 - val_command_output_accuracy: 0.9263 - val_command_output_1_accuracy: 0.1676 - val_participant_output_1_accuracy: 0.1031
Epoch 8/10
5/5 [==============================] - 2s 345ms/step - loss: 1.4330 - participant_output_loss: 0.3027 - command_output_loss: 1.1302 - command_output_1_loss: 3.4170e-05 - participant_output_1_loss: 1.5950e-05 - participant_output_accuracy: 1.0000 - command_output_accuracy: 0.9967 - c

Epoch 9/10
5/5 [==============================] - 2s 306ms/step - loss: 2.0344 - participant_output_loss: 0.5887 - command_output_loss: 1.4457 - command_output_1_loss: 5.6059e-05 - participant_output_1_loss: 4.1397e-05 - participant_output_accuracy: 0.9333 - command_output_accuracy: 0.9767 - command_output_1_accuracy: 0.0033 - participant_output_1_accuracy: 0.1100 - val_loss: 2.1941 - val_participant_output_loss: 0.6815 - val_command_output_loss: 1.5125 - val_command_output_1_loss: 5.0758e-05 - val_participant_output_1_loss: 5.2917e-05 - val_participant_output_accuracy: 0.8840 - val_command_output_accuracy: 0.8877 - val_command_output_1_accuracy: 0.0000e+00 - val_participant_output_1_accuracy: 0.1050
Epoch 10/10
5/5 [==============================] - 2s 352ms/step - loss: 1.9202 - participant_output_loss: 0.5376 - command_output_loss: 1.3826 - command_output_1_loss: 5.4357e-05 - participant_output_1_loss: 3.5406e-05 - participant_output_accuracy: 0.9400 - command_output_accuracy: 0.990

...

Epoch 5/10
5/5 [==============================] - 1s 277ms/step - loss: 2.5249 - participant_output_loss: 0.8907 - command_output_loss: 1.6339 - command_output_1_loss: 5.9404e-05 - participant_output_1_loss: 1.9624e-04 - participant_output_accuracy: 0.8033 - command_output_accuracy: 0.8433 - command_output_1_accuracy: 0.1133 - participant_output_1_accuracy: 0.2900 - val_loss: 2.5610 - val_participant_output_loss: 0.9117 - val_command_output_loss: 1.6491 - val_command_output_1_loss: 4.8885e-05 - val_participant_output_1_loss: 1.5546e-04 - val_participant_output_accuracy: 0.8103 - val_command_output_accuracy: 0.7680 - val_command_output_1_accuracy: 0.2781 - val_participant_output_1_accuracy: 0.5506
Epoch 6/10
5/5 [==============================] - 1s 281ms/step - loss: 2.3443 - participant_output_loss: 0.7906 - command_output_loss: 1.5536 - command_output_1_loss: 4.6132e-05 - participant_output_1_loss: 1.2254e-04 - participant_output_accuracy: 0.8633 - command_output_accuracy: 0.8400 - c

Epoch 7/10
5/5 [==============================] - 1s 281ms/step - loss: 1.0073 - participant_output_loss: 0.1726 - command_output_loss: 0.8346 - command_output_1_loss: 4.6228e-05 - participant_output_1_loss: 3.1047e-05 - participant_output_accuracy: 1.0000 - command_output_accuracy: 1.0000 - command_output_1_accuracy: 0.0200 - participant_output_1_accuracy: 0.2733 - val_loss: 1.3116 - val_participant_output_loss: 0.3380 - val_command_output_loss: 0.9735 - val_command_output_1_loss: 4.7497e-05 - val_participant_output_1_loss: 4.0717e-05 - val_participant_output_accuracy: 0.9521 - val_command_output_accuracy: 0.9319 - val_command_output_1_accuracy: 0.0129 - val_participant_output_1_accuracy: 0.1842
Epoch 8/10
5/5 [==============================] - 2s 331ms/step - loss: 0.9417 - participant_output_loss: 0.1565 - command_output_loss: 0.7851 - command_output_1_loss: 4.5048e-05 - participant_output_1_loss: 2.9804e-05 - participant_output_accuracy: 1.0000 - command_output_accuracy: 1.0000 - c

Epoch 9/10
5/5 [==============================] - 1s 293ms/step - loss: 2.1933 - participant_output_loss: 0.5239 - command_output_loss: 1.6692 - command_output_1_loss: 5.7111e-05 - participant_output_1_loss: 3.4824e-05 - participant_output_accuracy: 0.9433 - command_output_accuracy: 0.9000 - command_output_1_accuracy: 0.9967 - participant_output_1_accuracy: 0.2633 - val_loss: 2.3338 - val_participant_output_loss: 0.6280 - val_command_output_loss: 1.7057 - val_command_output_1_loss: 4.6071e-05 - val_participant_output_1_loss: 4.1446e-05 - val_participant_output_accuracy: 0.9116 - val_command_output_accuracy: 0.8306 - val_command_output_1_accuracy: 0.9319 - val_participant_output_1_accuracy: 0.3610
Epoch 10/10
5/5 [==============================] - 1s 293ms/step - loss: 2.0997 - participant_output_loss: 0.4715 - command_output_loss: 1.6281 - command_output_1_loss: 3.8774e-05 - participant_output_1_loss: 3.0207e-05 - participant_output_accuracy: 0.9733 - command_output_accuracy: 0.9000 - 

Epoch 1/10
5/5 [==============================] - 3s 586ms/step - loss: 4.1259 - participant_output_loss: 1.8310 - command_output_loss: 2.2641 - command_output_1_loss: 0.0017 - participant_output_1_loss: 0.0291 - participant_output_accuracy: 0.3500 - command_output_accuracy: 0.2700 - command_output_1_accuracy: 0.0000e+00 - participant_output_1_accuracy: 0.4267 - val_loss: 3.5667 - val_participant_output_loss: 1.4876 - val_command_output_loss: 2.0653 - val_command_output_1_loss: 5.0398e-04 - val_participant_output_1_loss: 0.0133 - val_participant_output_accuracy: 0.3444 - val_command_output_accuracy: 0.4678 - val_command_output_1_accuracy: 0.0000e+00 - val_participant_output_1_accuracy: 0.0000e+00
Epoch 2/10
5/5 [==============================] - 1s 297ms/step - loss: 3.3278 - participant_output_loss: 1.3880 - command_output_loss: 1.9295 - command_output_1_loss: 2.2712e-04 - participant_output_1_loss: 0.0101 - participant_output_accuracy: 0.5467 - command_output_accuracy: 0.5767 - comma

Epoch 3/10
5/5 [==============================] - 1s 217ms/step - loss: 1.4305 - participant_output_loss: 0.4276 - command_output_loss: 1.0028 - command_output_1_loss: 4.1972e-05 - participant_output_1_loss: 4.1546e-05 - participant_output_accuracy: 0.9933 - command_output_accuracy: 0.9967 - command_output_1_accuracy: 0.2333 - participant_output_1_accuracy: 0.3433 - val_loss: 1.6956 - val_participant_output_loss: 0.5722 - val_command_output_loss: 1.1233 - val_command_output_1_loss: 3.9479e-05 - val_participant_output_1_loss: 4.2268e-05 - val_participant_output_accuracy: 0.9337 - val_command_output_accuracy: 0.9171 - val_command_output_1_accuracy: 0.2044 - val_participant_output_1_accuracy: 0.2247
Epoch 4/10
5/5 [==============================] - 1s 288ms/step - loss: 1.3473 - participant_output_loss: 0.4021 - command_output_loss: 0.9451 - command_output_1_loss: 4.0563e-05 - participant_output_1_loss: 3.7555e-05 - participant_output_accuracy: 1.0000 - command_output_accuracy: 1.0000 - c

In [27]:
resnet_model = tf.keras.models.load_model('Initial_group_model_size6_0608.h5')
predictions = resnet_model.predict(Test_Inputs)[0]
predicted_classes = np.argmax(predictions, axis=1)+1
acc_c = round(sum(x == y for x, y in zip(test_unit_participant_class, predicted_classes)) / len(test_unit_participant_class),4)

predictions = resnet_model.predict(Test_Inputs)[1]
predicted_classes = np.argmax(predictions, axis=1)
acc_s = round(sum(x == y for x, y in zip(test_unit_command_class, predicted_classes)) / len(test_unit_command_class),4)
overall_acc = acc_c + acc_s
print(acc_c,acc_s)

0.9558 0.9319


In [28]:
Perfomance

,Model,Size,Time,Partcp_Acc_p15,Command_Acc_p15,Partcp_Acc_p1,Command_Acc_p1,Partcp_Acc_p2,Command_Acc_p2,Partcp_Acc_p3,Command_Acc_p3,Partcp_Acc_p4,Command_Acc_p4,Partcp_Acc_p5,Command_Acc_p5,Acc_p6,Acc_p7,Acc_p8
0,Group,10,47.540156,0.9724,0.9484,0.991,0.9550,0.9259,0.8796,0.9652,0.9565,0.9817,0.9633,1.00,0.99,0.67,0.528,0.3366
1,Group,10,48.307249,0.9779,0.9521,1.000,0.9369,0.9352,0.8796,0.9739,0.9652,0.9908,0.9908,0.99,0.99,0.59,0.528,0.2970
2,Group,10,46.201615,0.9613,0.9448,0.991,0.9369,0.9074,0.8611,0.9826,0.9652,0.9358,0.9725,0.99,0.99,0.67,0.432,0.2673
3,Group,10,43.692362,0.9650,0.9521,1.000,0.9459,0.9074,0.8704,0.9826,0.9652,0.9358,0.9908,1.00,0.99,0.62,0.504,0.2673
4,Group,10,44.504580,0.9687,0.9484,0.991,0.9550,0.9259,0.8796,0.9739,0.9652,0.9541,0.9541,1.00,0.99,0.59,0.472,0.3267
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75,Group,6,36.358289,0.9484,0.9116,0.991,0.9189,0.8611,0.7963,0.9826,0.9478,0.9083,0.9358,1.00,0.96,0.47,0.432,0.2871
76,Group,6,35.753378,0.9595,0.9134,0.991,0.9189,0.9167,0.7870,0.9739,0.9478,0.9174,0.9633,1.00,0.95,0.45,0.416,0.3168
77,Group,6,35.383533,0.9558,0.9319,0.982,0.9459,0.9074,0.8519,0.9652,0.9391,0.9266,0.9541,1.00,0.97,0.56,0.480,0.3267
78,Group,6,35.488919,0.9576,0.9116,0.982,0.9369,0.9259,0.7963,0.9565,0.9217,0.9358,0.9358,0.99,0.97,0.54,0.496,0.2970


In [29]:
Perfomance.to_csv('Performance_0608_training_Data_Size.csv',index=False)

In [30]:
best_index

2